<a href="https://colab.research.google.com/github/NoahLee99/ML-DL-studylog/blob/main/Chapter%2009-1%20-%20RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **순차 데이터와 순환 신경망**

**키워드**

* 순차 데이터: 텍스트나 시계열 데이터와 같이 순서에 의미가 있는 데이터이다.
<br> 대표적인 순차 데이터로는 글, 대화, 일자별 날씨, 일자별 판매 실적 등을 예로 들 수 있다.

* 순환 신경망: 순차 데이터에 잘 맞는 인공 신경망의 한 종류이다.
<br> 순차 데이터를 처리하기 위해 고안된 순환층을 1개 이상 사용한 신경망을 순환 신경망이라 부른다.

* 셀: 순환신경망에서 종종 순환층을 셀이라 부른다
<br> 일반적인 인공 신경망과 마찬가지로 하나의 셀은 여러 개의 뉴런으로 구성된다.

* 은닉 상태: 순환 신경망에서는 셀의 출력을 특별히 은닉 상태라고 부른다.
<br> 은닉 상태는 다음 층으로 전달될 뿐만 아니라 셀이 다음 타임스텝의 데이터를 처리할 때 재사용된다.

In [ ]:
'''
내가 가장 관심 있고 궁금한 "자연어 처리(NLP)"를 위한 RNN을 배울 차례이다.

현재 한빛 마켓은 고객들의 상품 평가를 분석하기로 결정한 상황이다.
즉 상품 페이지의 댓글들을 확인하여 반응을 살피기로 한 것이다.
그러나 이 리뷰들을 육안으로 일일이 확인하는 것은 불가능할 것이다.

과연 딥러닝을 통해 댓글을 분석하여 리뷰(평가)가 긍정적인지 부정적인지 판단할 수 있을까?
'''

# **순차 데이터**

In [ ]:
'''
"순차 데이터(sequential data)"는 텍스트나
"시계열 데이터(time series data)(=일정한 시간 간격으로 기록된 데이터)"와 같이 순서에 의미가 있는 데이터를 말한다.
예를 들어 "I am a boy"는 쉽게 이해할 수 있지만 "boy am a I"는 말이 되지 않는다.
또 일별 온도를 기록한 데이터에서 날짜 순서를 뒤죽박죽 섞는다면 내일의 온도를 쉽게 예상하기 어려울 것이다.

지금까지 우리가 보았던 데이터들은 순서와는 상관이 없었다.
일례로 패션 MNIST 데이터를 생각해 보자.
이 데이터를 신경망 모델에 전달할 때 샘플을 랜덤하게 섞은 후 훈련 세트와 검증 세트로 나누었다.
즉 샘플의 순서와 전혀 상관이 없었다.
심지어 골고루 섞는 편이 결과가 더 좋다.
이는 딥러닝뿐만 아니라 일반적인 머신러닝 모델에서도 마찬가지이다.
4장에서 봤던 생선 데이터나 패션 MNIST 데이터는 어떤 샘플이 먼저 주입되어도 모델의 학습에 큰 영향을 미치지 않는다.

이번 장에서 사용하려는 댓글, 즉 텍스트 데이터는 단어의 순서가 중요한 순차 데이터이다.
이런 데이터는 순서를 유지하며 신경망에 주입해야 한다.
단어의 순서를 마구 섞어서 주입하면 안 된다.

따라서 순차 데이터를 다룰 땐 이전에 입력한 데이터를 기억하는 능력이 필요하다.
예를 들어 "별로지만 추천해요"에서 "추천해요"가 입력될 때 "별로지만"을 기억하고 있어야
이 댓글을 무조건 긍정적이라고만 판단하지 않을 것이다.

완전 연결 신경망이나 합성곱 신경망은 이런 기억 장치가 없다.
하나의 샘플(또는 하나의 배치)을 사용하여 정방향 계산을 수행하고 나면
그 샘플은 버려지고 다음 샘플을 처리할 때 재사용하지 않는다.

이렇게 이렇게 입력 데이터의 흐름이 앞으로만 전달되는 신경망을 "피드포워드 신경망(feedforward neural network, FFNN)"이라고 한다.
이전 장에서 배웠던 완전 연결 신경망과 합성곱 신경망이 모두 피드포워드 신경망에 속한다.

신경망이 이전에 처리했떤 샘플을 다음 샘플을 처리하는데 재사용하기 위해서는 이렇게 데이터 흐름이 앞으로만 전달되어서는 곤란하다.
다음 샘플을 위해서 이전 데이터가 신경망 층에 순환될 필요가 있다.
이런 신경망이 바로 순환 신경망이다.
'''

'\n"순차 데이터(sequential data)"는 텍스트나 \n"시계열 데이터(time series data)(=일정한 시간 간격으로 기록된 데이터)"와 같이 순서에 의미가 있는 데이터를 말한다.\n예를 들어 "I am a boy"는 쉽게 이해할 수 있지만 "boy am a I"는 말이 되지 않는다.\n또 일별 온도를 기록한 데이터에서 날짜 순서를 뒤죽박죽 섞는다면 내일의 온도를 쉽게 예상하기 어려울 것이다.\n\n지금까지 우리가 보았던 데이터들은 순서와는 상관이 없었다.\n일례로 패션 MNIST 데이터를 생각해 보자.\n이 데이터를 신경망 모델에 전달할 때 샘플을 랜덤하게 섞은 후 훈련 세트와 검증 세트로 나누었다.\n즉 샘플의 순서와 전혀 상관이 없었다.\n심지어 골고루 섞는 편이 결과가 더 좋다.\n이는 딥러닝뿐만 아니라 일반적인 머신러닝 모델에서도 마찬가지이다.\n4장에서 봤던 생선 데이터나 패션 MNIST 데이터는 어떤 샘플이 먼저 주입되어도 모델의 학습에 큰 영향을 미치지 않는다.\n\n이번 장에서 사용하려는 댓글, 즉 텍스트 데이터는 단어의 순서가 중요한 순차 데이터이다.\n이런 데이터는 순서를 유지하며 신경망에 주입해야 한다.\n단어의 순서를 마구 섞어서 주입하면 안 된다.\n\n따라서 순차 데이터를 다룰 땐 이전에 입력한 데이터를 기억하는 능력이 필요하다.\n예를 들어 "별로지만 추천해요"에서 "추천해요"가 입력될 때 "별로지만"을 기억하고 있어야 \n이 댓글을 무조건 긍정적이라고만 판단하지 않을 것이다.\n\n완전 연결 신경망이나 합성곱 신경망은 이런 기억 장치가 없다.\n하나의 샘플(또는 하나의 배치)을 사용하여 정방향 계산을 수행하고 나면 \n그 샘플은 버려지고 다음 샘플을 처리할 때 재사용하지 않는다.\n\n이렇게 이렇게 입력 데이터의 흐름이 앞으로만 전달되는 신경망을 "피드포워드 신경망(feedforward neural network, FFNN)"이라고 한다.\n이전 장에서 배웠던 완전 연결 신경망과 합성곱 신경망이 모두 피드포워드

# **순환 신경망**

In [ ]:
'''
"순환 신경망(recurrent neural network, RNN)"은 일반적인 완전 연결 신경망과 거의 비슷하다.
완전 연결 신경망에 이전 데이터의 처리 흐름을 순환하는 고리 하나만 추가하면 된다!
즉 뉴런의 출력이 다시 자기 자신으로 전달되는 것이다.
어떤 샘플을 처리할 때 바로 이전에 사용했던 데이터를 재사용하는 셈이다.
조금 더 간단한 예를 들어 보자.
a, b, c 3개의 샘플을 처리하는 순환 신경망의 뉴런이 있다고 가정해 보자.
첫 번째 샘플 a를 처리하고 난 출력 R(recurrent)a가 다시 뉴런으로 들어간다.
이 출력 Ra에는 a에 대한 정보가 다분히 들어 있다.
그다음 b를 처리할 때 앞에서 a를 사용해 만든 Ra를 함께 사용한다.
따라서 Ra와 b를 사용해서 만든 Rb에는 A에 대한 정보가 어느 정도 포함되어 있을 것이다.
그다음 c를 처리할 때는 동일한 방식으로 Rb를 함께 사용한다.

이렇게 Rb와 c를 사용해 만든 Rc에는 어떤 정보들이 포함되어 있을까?
Rb를 사용했으므로 당연히 b에 대한 정보가 어느 정도 포함되어 있을 것이고,
Rb에는 a에 대한 정보도 포함되어 있을 것이다.
따라서 Rc에 b와 a에 대한 정보가 담겨 있다고 말할 수 있다.

물론 Rc에는 a에 대한 정보보다는 b에 대한 정보가 더 많다.
그래서 순환 신경망에서는 "이전 샘플에 대한 기억을 가지고 있다"고 종종 말한다.
이렇게 샘플을 처리하는 한 단계를 "타임스텝(timestep)"이라고 말한다.

순환 신경망은 이전 타임스텝의 샘플을 기억하지만 타임스텝이 오래될수록 순환되는 정보는 희미해진다.
나중에 여기에 대해 다시 언급하겠다.

순환 신경망에서는 특별히 층을 "셀(cell)"이라고 부른다.
한 셀에는 여러 개의 뉴런이 있지만 완전 연결 신경망과 달리 뉴런을 모두 표시하지 않고 하나의 셀로 층을 표현한다.
또 셀의 출력을 "은닉 상태(hidden state)"라고 부른다.

합성곱 신경망에서처럼 신경망의 구조마다 조금씩 부르는 이름이 다를 수 있다.
하지만 기본 구조는 같다.
입력에 어떤 가중치를 곱하고 활성화 함수를 통과시켜 다음 층으로 보내는 것이다.
달라지는 것은 층의 출력(즉 은닉 상태)을 다음 타임스텝에 재사용한다는 것뿐이다.

일반적으로 은닉층의 활성화 함수로는 하이퍼볼릭 탄젠트(hyperbolic tangent) 함수인 tanh^2가 많이 사용된다.
tanh 함수도 S자 모양을 띠기 때문에 종종 시그모이드 함수라고 부르기도 한다.
하지만 헷갈림을 방지하고자 이렇게 부르지 않겠다.
tanh 함수는 시그모이드 함수와는 달리 -1~1 사이의 범위를 거진다. (시그모이드 함수는 0~1 사이의 범위)

다른 신경망과 마찬가지로 순환 신경망 그림에도 번거로움을 피하기 위해 활성화 함수를 표시하지 않는 경우가 많다.
하지만 순환 신경망에서도 활성화 함수가 반드시 필요하다는 것은 꼭 기억하자.

합성곱 신경망과 같은 같은 피드포워드 신경망에서 뉴런은 입력과 가중치를 곱한다.
순환 신경망에서도 동일하다.
다만 순환 신경망의 뉴런은 가중치가 하나 더 있다.
바로 이전의 타임스텝의 은닉 상태에 곱해지는 가중치이다. (b를 처리할 때 Ra를 사용)
셀은 입력과 이전 타임스텝의 은닉 상태를 사용하여 현재 타임스텝의 은닉 상태를 만든다.
(피프포워드 신경망과 마찬가지로 뉴런마다 하나의 절편이 포함된다.)

순환 신경망을 타임스텝마다 그릴 수 있는데, 이런 그림을 보고 셀을 타임스텝으로 펼쳤다고 말한다.
타임스텝 1에서 셀의 출력 h1이 다임스텝 2의 셀로 주입되며 이때 Wh와 곱해진다.
마찬가지로 타입스텝 2에서 셀의 출력 h2가 타임스텝 3의 셀로 주입되며 이때에도 Wh와 곱해진다.

여기서 알 수 있는 것은 모든 타임스텝에서 사용되는 가중치는 Wh 하나라는 점이다.
가중치 Wh는 타임스텝에 따라 변화되는 뉴런의 출력을 학습한다.
이런 능력이 이절의 시작 부분에 언급했던 순차 데이터를 다루는 데 필요한다.

그럼 맨 처음 타임스텝 1에서 사용되는 이전 은닉 상태 h0은 어떻게 구할 수 있을까?
맨 처음 샘플을 입력할 때는 이전 타임스텝이 없다.
따라서 간단히 h0은 모두 0으로 초기화한다.
'''

'\n"순환 신경망(recurrent neural network, RNN)"은 일반적인 완전 연결 신경망과 거의 비슷하다.\n완전 연결 신경망에 이전 데이터의 처리 흐름을 순환하는 고리 하나만 추가하면 된다!\n즉 뉴런의 출력이 다시 자기 자신으로 전달되는 것이다.\n어떤 샘플을 처리할 때 바로 이전에 사용했던 데이터를 재사용하는 셈이다.\n조금 더 간단한 예를 들어 보자.\na, b, c 3개의 샘플을 처리하는 순환 신경망의 뉴런이 있다고 가정해 보자.\n첫 번째 샘플 a를 처리하고 난 출력 R(recurrent)a가 다시 뉴런으로 들어간다.\n이 출력 Ra에는 a에 대한 정보가 다분히 들어 있다.\n그다음 b를 처리할 때 앞에서 a를 사용해 만든 Ra를 함께 사용한다.\n따라서 Ra와 b를 사용해서 만든 Rb에는 A에 대한 정보가 어느 정도 포함되어 있을 것이다.\n그다음 c를 처리할 때는 동일한 방식으로 Rb를 함께 사용한다.\n\n이렇게 Rb와 c를 사용해 만든 Rc에는 어떤 정보들이 포함되어 있을까?\nRb를 사용했으므로 당연히 b에 대한 정보가 어느 정도 포함되어 있을 것이고,\nRb에는 a에 대한 정보도 포함되어 있을 것이다.\n따라서 Rc에 b와 a에 대한 정보가 담겨 있다고 말할 수 있다.\n\n물론 Rc에는 a에 대한 정보보다는 b에 대한 정보가 더 많다.\n그래서 순환 신경망에서는 "이전 샘플에 대한 기억을 가지고 있다"고 종종 말한다.\n이렇게 샘플을 처리하는 한 단계를 "타임스텝(timestep)"이라고 말한다.\n\n순환 신경망은 이전 타임스텝의 샘플을 기억하지만 타임스텝이 오래될수록 순환되는 정보는 희미해진다.\n나중에 여기에 대해 다시 언급하겠다.\n\n순환 신경망에서는 특별히 층을 "셀(cell)"이라고 부른다.\n한 셀에는 여러 개의 뉴런이 있지만 완전 연결 신경망과 달리 뉴런을 모두 표시하지 않고 하나의 셀로 층을 표현한다.\n또 셀의 출력을 "은닉 상태(hidden state)"라고 부른다.\n\n합성곱 신경망에서처럼 신경망

# **셀의 가중치와 입출력**

In [ ]:
'''
순환 신경망의 셀에서 필요한 가중치 크기를 계산해 보겠다.
복잡한 모델을 배울수록 가중치 개수를 계산해 보면 잘 이해하고 있는지 알 수 있다.
예를 들어 순환층에 입력되는 특성의 개수가 4개이고 순환층의 뉴런이 3개(r1, r2, r3)라고 가정해 보겠다.
먼저 Wx(원래 입력에 곱해지는 가중치)의 크기를 구해 보자.
입력층과 순환층의 뉴런이 모두 완전 연결되기 때문에 가중치 Wx의 크기는 4 * 3 = 12개가 된다.
7장에서 본 완전 연결 신경망의 입력층과 은닉층의 연결과 같다.

그럼 순환층에서 다음 타임스텝에 재사용되는 은닉 상태를 위한 가중치 Wh의 크기는 어떻게 될까?
순환층에 있는 첫 번째 뉴런(r1)의 은닉 상태가 다음 타임스텝에 재사용될 때
첫 번째 뉴런과 두 번째 뉴런, 세 번째 뉴런에 모두 전달된다.
즉 이전 타임스텝의 은닉 상태는 다음 타임스텝의 뉴런에 완전히 연결된다!

두 번째 뉴런(r2)의 은닉 상태도 마찬가지로 첫 번째 뉴런과 두 번째 뉴런, 세 번째 뉴런에 모두 전달된다.
따라서 이 순환층에서 은닉 상태를 위한 가중치 Wh는 3 * 3 = 9개이다. (rn 하나당 3번씩 계산되므로)

가중치는 모두 구했으니 모델 파라미터 개수를 계산해 보자.
간단하게 가중치에 절편을 더하면 된다.

* 모델 파라미터 개수는 모델이 학습해야 할 가중치와 편향을 의미하며,
  모델의 복잡도와 표현력을 나타내는 중요한 지표이다.

여기엔 각 뉴런마다 하나의 절편이 있다.
따라서 이 순환층은 모두 12 + 9 + 3 = 24개의 모델 파라미터를 가지고 있다.
이제 왜 순환층을 셀 하나로 표시할 수밖에 없는지 이해가 되는가?
은닉 상태가 모든 뉴런에 순환되기 때문에 완전 연결 신경망처럼 그림으로 표현하기는 너무 어렵다.

모델 파라미터 수 = Wx + Wh + 절편 = 12 + 9 + 3 = 24
'''

'\n순환 신경망의 셀에서 필요한 가중치 크기를 계산해 보겠다.\n복잡한 모델을 배울수록 가중치 개수를 계산해 보면 잘 이해하고 있는지 알 수 있다.\n예를 들어 순환층에 입력되는 특성의 개수가 4개이고 순환층의 뉴런이 3개(r1, r2, r3)라고 가정해 보겠다.\n먼저 Wx(원래 입력에 곱해지는 가중치)의 크기를 구해 보자.\n입력층과 순환층의 뉴런이 모두 완전 연결되기 때문에 가중치 Wx의 크기는 4 * 3 = 12개가 된다.\n7장에서 본 완전 연결 신경망의 입력층과 은닉층의 연결과 같다.\n\n그럼 순환층에서 다음 타임스텝에 재사용되는 은닉 상태를 위한 가중치 Wh의 크기는 어떻게 될까?\n순환층에 있는 첫 번째 뉴런(r1)의 은닉 상태가 다음 타임스텝에 재사용될 때 \n첫 번째 뉴런과 두 번째 뉴런, 세 번째 뉴런에 모두 전달된다.\n즉 이전 타임스텝의 은닉 상태는 다음 타임스텝의 뉴런에 완전히 연결된다!\n\n두 번째 뉴런(r2)의 은닉 상태도 마찬가지로 첫 번째 뉴런과 두 번째 뉴런, 세 번째 뉴런에 모두 전달된다.\n따라서 이 순환층에서 은닉 상태를 위한 가중치 Wh는 3 * 3 = 9개이다. (rn 하나당 3번씩 계산되므로)\n\n가중치는 모두 구했으니 모델 파라미터 개수를 계산해 보자.\n간단하게 가중치에 절편을 더하면 된다.\n\n* 모델 파라미터 개수는 모델이 학습해야 할 가중치와 편향을 의미하며, \n  모델의 복잡도와 표현력을 나타내는 중요한 지표이다.\n\n여기엔 각 뉴런마다 하나의 절편이 있다.\n따라서 이 순환층은 모두 12 + 9 + 3 = 24개의 모델 파라미터를 가지고 있다.\n이제 왜 순환층을 셀 하나로 표시할 수밖에 없는지 이해가 되는가?\n은닉 상태가 모든 뉴런에 순환되기 때문에 완전 연결 신경망처럼 그림으로 표현하기는 너무 어렵다.\n\n모델 파라미터 수 = Wx + Wh + 절편 = 12 + 9 + 3 = 24\n'

In [ ]:
'''
순환층의 가중치 크기를 알아보았으므로 이번에는 순환층의 입력과 출력에 대해 생각해 보자.
이전 장에서 배웠던 합성곱 층의 입력은 전형적으로 하나의 샘플이 3개의 차원을 가진다.
너비, 높이, 채널이다. (RGB 컬러 이미지)
입력이 합성곱 층과 풀링 층을 통과하면 너비, 높이, 채널(혹은 깊이)의 크기가 달라지지만 차원의 개수는 그대로 유지되었다.

순환층은 일반적으로 샘플마다 2개의 차원을 가진다.
보통 하나의 샘플을 하나의 "시퀀스(sequence)"라고 말한다.
시퀀스 안에는 여러 개의 아이템이 들어 있다.
여기에서 시퀀스의 길이가 바로 타임스텝 길이가 된다.

예를 들어 어떤 샘플에 "I am a boy"란 문장이 들어 있다고 가정해 보자.
이 샘플은 4개의 단어로 이루어져 있다.
각 단어를 3개의 어떤 숫자로 표현한다고 가정해 보자. (이 숫자 표현은 다음 장에서 자세히 다룬다)
이는 (1, 4, 3)의 타임스텝 크기를 가진다. (미니배치 사이즈, 시퀀스 수, 시퀀스 길이)

이런 입력이 순환층을 통과하면 두 번째, 세 번째 차원이 사라지고 순환층의 뉴런 개수만큼 출력된다.
이를 차근차근 설명해 보겠다.
하나의 샘플은 시퀀스 길이(여기에서는 단어 개수)와 단어 표현(다음 장에서 자세히 다룬다)의 2차원 배열이다.
순환층을 통과하면  1차원 배열로 바뀐다.
이 1차원 배열의 크기는 순환층의 뉴런 개수에 의해 결정된다.

이 설명이 의아하게 느껴지지 않는가?
앞에서 셀의 출력을 설명할 때 빠뜨린 것이 있기 때문이다.
앞에서는 셀이 모든 타임스텝에서 출력을 만든 것처럼 표현했다.
하지만 사실 순환층은 기본적으로 마지막(최신) 타임스텝의 은닉 상태만 출력으로 내보낸다.

이는 마치 입력된 시퀀스 길이를 모두 읽어서 정보를 마지막 은닉 상태에 압축하여 전달하는 것이다.
이제 왜 순환 신경망이 정보를 기억하는 메모리를 가진다고 표현하는지 이해할 수 있다.
또 순환 신경망이 순차 데이터에 잘 맞는 이유를 파악할 수 있다.

순환 신경망도 완전 연결 신경망이나 합성곱 신경망처럼 여러 개의 층을 쌓을 수 있다.
순환층을 여러 개 쌓았을 때는 셀의 출력이 어떻게 달라질까?
셀의 입력은 샘플마다 타임스텝과 단어 표현으로 이루어진 2차원 배열이어야 한다.
따라서 첫 번째 셀이 마지막 타임스텝의 은닉 상태만 출력해서는 안 될 것이다.
이런 경우에는 마지막 셀을 제외한 다른 모든 셀은 모든 타임스텝의 은닉 상태를 출력한다.
예를 들어 2개의 순환층을 쌓은 경우에는,
첫 번째 셀은 모든 타임스텝의 은닉 상태를 출력하고, 두 번째 셀은 마지막 타임스텝의 은닉 상태만 출력한다.
다음 장에서 이런 예를 직접 다루어 보겠다.

마지막으로 출력층의 구성에 대해 알아보겠다.
합성곱 신경망과 마찬가지로 순환 신경망도 마지막에는 밀집층을 두어 클래스를 분류한다.
다중 분류일 경우에는 출력층에 클래스 개수만큼 뉴런을 두고 소프트맥스 활성화 함수를 사용한다.
이진 분류일 경우에는 하나의 뉴런을 두고 시그모이드 활성화 함수를 사용한다.

합성곱 신경망과 다른 점은 마지막 셀의 출력이 1차원이기 때문에 Flatten 클래스로 펼칠 필요가 없다.
셀의 출력을 그대로 밀집층에 사용할 수 있다.
예를 들어 다중 분류 문제에서 입력 샘플의 크기가 (20, 100)일 경우라고 생각해 보자.
여기서 샘플은 20개의 타임으로 이러우져 있다.
또 각 타임스텝은 100개의 표현 또는 특성으로 이루어져 있다. (타임스텝의 길이가 100)
이 샘플이 순환층의 셀을 통과하면 모든 타임스텝을 처리하고 난 후의 은닉 상태만 출력된다.
이 은닉 상태의 크기는 셀에 있는 뉴런의 개수가 되므로 (10, )이다. (뉴런 10개의 순환층을 통과한 출력)

샘플마다 셀이 1차원 배열을 출력하기 때문에 합성곱 신경망처럼 Flatten 클래스로 펼칠 필요 없이 바로 출력층에 연결 가능하다.
'''

'\n순환층의 가중치 크기를 알아보았으므로 이번에는 순환층의 입력과 출력에 대해 생각해 보자.\n이전 장에서 배웠던 합성곱 층의 입력은 전형적으로 하나의 샘플이 3개의 차원을 가진다.\n너비, 높이, 채널이다. (RGB 컬러 이미지)\n입력이 합성곱 층과 풀링 층을 통과하면 너비, 높이, 채널(혹은 깊이)의 크기가 달라지지만 차원의 개수는 그대로 유지되었다.\n\n순환층은 일반적으로 샘플마다 2개의 차원을 가진다.\n보통 하나의 샘플을 하나의 "시퀀스(sequence)"라고 말한다.\n시퀀스 안에는 여러 개의 아이템이 들어 있다.\n여기에서 시퀀스의 길이가 바로 타임스텝 길이가 된다.\n\n예를 들어 어떤 샘플에 "I am a boy"란 문장이 들어 있다고 가정해 보자.\n이 샘플은 4개의 단어로 이루어져 있다.\n각 단어를 3개의 어떤 숫자로 표현한다고 가정해 보자. (이 숫자 표현은 다음 장에서 자세히 다룬다)\n이는 (1, 4, 3)의 타임스텝 크기를 가진다. (미니배치 사이즈, 시퀀스 수, 시퀀스 길이)\n\n이런 입력이 순환층을 통과하면 두 번째, 세 번째 차원이 사라지고 순환층의 뉴런 개수만큼 출력된다.\n이를 차근차근 설명해 보겠다.\n하나의 샘플은 시퀀스 길이(여기에서는 단어 개수)와 단어 표현(다음 장에서 자세히 다룬다)의 2차원 배열이다.\n순환층을 통과하면  1차원 배열로 바뀐다.\n이 1차원 배열의 크기는 순환층의 뉴런 개수에 의해 결정된다.\n\n이 설명이 의아하게 느껴지지 않는가?\n앞에서 셀의 출력을 설명할 때 빠뜨린 것이 있기 때문이다.\n앞에서는 셀이 모든 타임스텝에서 출력을 만든 것처럼 표현했다.\n하지만 사실 순환층은 기본적으로 마지막(최신) 타임스텝의 은닉 상태만 출력으로 내보낸다.\n\n이는 마치 입력된 시퀀스 길이를 모두 읽어서 정보를 마지막 은닉 상태에 압축하여 전달하는 것이다.\n이제 왜 순환 신경망이 정보를 기억하는 메모리를 가진다고 표현하는지 이해할 수 있다.\n또 순환 신경망이 순차 데이터에 잘 맞는 이유를 파악